In [1]:
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPool2D, Flatten, Dense,Dropout
from tensorflow.keras.models import Model, load_model, save_model

from tensorflow.keras.callbacks import EarlyStopping ,CSVLogger
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from math import ceil
import pandas as pd 
import cv2 as cv
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn as sk
import keras 
from tensorflow.keras import datasets, layers, models
%matplotlib inline

ModuleNotFoundError: No module named 'sklearn'

In [ ]:


def load_data(dir_list, image_size):


    # load all images in a directory
    data = []
    label = []
    image_width, image_height = image_size
    
    for directory in dir_list:
        for filename in os.listdir(directory):
            
            image = cv.imread(directory + '/' + filename,0)

            image = cv.resize(image, dsize=(image_width, image_height), interpolation=cv.INTER_CUBIC)

            image = image / 255.0
            image= image.astype(np.float32)

            data.append(image)

            if directory == 'yes':
                label.append([1]) ## have tumor
                
            else:
                label.append([0]) ## donot have tumor 
            

    data_array = np.array(data)
    label = np.array(label) 
    return data_array, label


        


In [ ]:
x,y=load_data(['yes','no'],(256,256))
# print(x[2])
print(y[2])
plt.imshow(x[2], cmap='gray')
plt.axis('off')
plt.title('sample_image')

In [ ]:
def split_data(test_size=0.2 , log=True):
    x_train, x_test_val, y_train, y_test_val = train_test_split(x, y, test_size=test_size,shuffle=True)
    if log==True : 
        print('Input Shape : ')
        print(x_train.shape,(y_train).shape)
        print(x_test_val.shape,(y_test_val).shape)
    x_test=x_test_val
    x_val=x_test_val
    y_test=y_test_val
    y_val=y_test_val
    
        
    x_train=x_train.reshape(-1,256,256,1)  
#     y_train=y_train.reshape(-1,100,100,1)  
#     y_test=y_test.reshape(-1,100,100,1)    
    x_test=x_test.reshape(-1,256,256,1)
    x_val=x_val.reshape(-1,256,256,1)
    if log==True : 
        print('Output Shape : ')
        print(x_train.shape,y_train.shape)
        print(x_test.shape,y_test.shape)
        print(x_val.shape,y_val.shape)
    return x_train, x_test, y_train, y_test,x_val,y_val
x_train, x_test, y_train, y_test,x_val,y_val=split_data()
validation_dataset=(x_val,y_val)


In [ ]:
def Build_Model(input_shape=(256,256,1)):
    Model = keras.models.Sequential([
            Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=input_shape),
            Conv2D(32,kernel_size=(3,3),activation='relu'),
            MaxPool2D(2,2),
            Conv2D(32,kernel_size=(3,3),activation='relu'),    
            Conv2D(32,kernel_size=(3,3),activation='relu'),    
            Conv2D(64,kernel_size=(3,3),activation='relu'),
            MaxPool2D(4,4),
            Flatten() ,    
            Dense(64,activation='relu') ,        
            Dense(32,activation='relu') ,   
            Dense(16,activation='relu'),
            Dropout(rate=0.5) ,            
            Dense(1,activation='sigmoid') ,    
            ])
    return Model

In [ ]:
Model=Build_Model()
Model.summary()

In [ ]:
Model.compile(optimizer='adam',
              loss="binary_crossentropy",
              metrics=['accuracy'])

In [ ]:
# extra

# def make_callbacks(csv_path='my_class_log_5.csv'):
#     early_stop=EarlyStopping(monitor='val_loss',patience=8,mode='min',restore_best_weights=False)
#     csv_logger=CSVLogger(csv_path)
#     callbacks =[early_stop,csv_logger]
#     return callbacks
# callbacks=make_callbacks()

# epochs = 1
# Model.fit(x_train, y_train, epochs=epochs,batch_size=32,verbose=1,callbacks=callbacks,validation_data=validation_dataset)

In [ ]:
epochs = 30
Model.fit(x_train, y_train, epochs=epochs,batch_size=32,verbose=1,validation_data=validation_dataset)

In [ ]:
Model.save('tumor_classification_trained_model_by_parham_2.h5')

In [ ]:
Model = load_model('tumor_classification_trained_model_by_parham_2.h5')

In [ ]:
Model_loss, Model_accuracy = Model.evaluate(x_test, y_test)

print(f"it is Model accuracy: {Model_accuracy}")
print(f'it is Model loss: {Model_loss}')

In [ ]:
# extra


def predict_human_readable(array):
    array=array.flatten()
#     print(array[0])
    result=1 if array>0.5 else 0
    return result   
#----------------------------------------
def predict_on_custome_image(img_path,img_or_directory=False,plot=False):
    dict_1={}
    if img_or_directory==False :
        array=cv.imread(img_path,0)
        
        shape_array=array.shape
        array_1=cv.resize(array,(256,256))
        
        y,x=array_1.shape
#         array_1=array_1 / 255.0
        array=array_1.reshape(1,y,x,1)
        #-------
#         print(array.shape)
        y_predict=Model.predict(array)
        y_predict_res=predict_human_readable(y_predict)
        

        if y_predict_res== 1:
            predicted='Tumor Detected'
            
        else : 
            predicted='do not have tumor'
        
        dict_1.update({img_path:predicted})
        if plot==True :
            plt.imshow(array_1,cmap='gray')
            plt.title(predicted)
            plt.axis('off')
    else :
        all_img=os.listdir(img_path)
        image_count=len(all_img)
#         print(all_img)
        if plot==True :
            rows,columns=3,ceil(image_count/3)
            fig = plt.figure(figsize=(20, 14))
            
        for c,path in enumerate(all_img) :
            lst_1=path.split('.')
            if lst_1[1]=='jpg' or lst_1[1]=='png' or lst_1[1]=='jpeg': 
                array=cv.imread(img_path+'/'+path,0)
    #             print(img_path+'/'+path)
                shape_array=array.shape
                array_1=cv.resize(array,(256,256))
                y,x=array_1.shape
#                 array_1=array_1 / 255.0
#                 array_1= array_1.astype(np.float32)
                array=array_1.reshape(1,y,x,1)
                
            #-------

                y_predict=Model.predict(array)
                y_predict_res=predict_human_readable(y_predict)
                if y_predict_res==1 :
                    predicted='Tumor Detected'

                else : 
                    predicted='do not have tumor'
                dict_1.update({path:predicted})
                if plot==True :
                    fig.add_subplot(rows, columns, c+1)
                    plt.imshow(array_1,cmap='gray')
                    plt.title(predicted)
                    plt.axis('off')
            else :
                continue
    return dict_1           
a=predict_on_custome_image('sample_image',img_or_directory=True,plot=True)
print(a)

In [ ]:
import cv2
import os
import numpy as np
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt

def predict_human_readable(array):
    array = array.flatten()
    result = 1 if array > 0.5 else 0
    return result   

def predict_on_custom_image(model, plot=False):
    dict_1 = {}
    
    # Browse for an image file
    root = tk.Tk()
    root.withdraw()  # Hide the main tkinter window
    file_path = filedialog.askopenfilename(title="Select Image File", filetypes=(("Image files", "*.jpg;*.jpeg;*.png;*.bmp"), ("All files", "*.*")))
    
    if file_path:
        img = cv2.imread(file_path, 0)
        array_1 = cv2.resize(img, (256, 256))
        y, x = array_1.shape
        array = array_1.reshape(1, y, x, 1)

        y_predict = model.predict(array)
        y_predict_res = predict_human_readable(y_predict)

        if y_predict_res == 1:
            predicted = 'Tumor Detected'
        else:
            predicted = 'No tumor detected'
        
        dict_1[file_path] = predicted
        
        if plot:
            plt.imshow(array_1, cmap='gray')
            plt.title(predicted)
            plt.axis('off')
            plt.show()

    return dict_1   

# Example usage
# Assuming 'Model' is your trained model
result = predict_on_custom_image(Model, plot=True)
print(result)
